In [3]:
import psycopg2
import pandas as pd
from io import StringIO
import csv

In [5]:
try:
    conn=psycopg2.connect(
        host=input("hostname: "),
        database=input("database: "),
        port=eval(input("port: ")),
        user=input("user: "),
        password=input("password: "))
    print(f"Connection successful...")
except Exception as e:
    print(f"{e}")
        

hostname: localhost
database: healthcare_data
port: 5432
user: postgres
password: Hewasborn@1889
Connection successful...


In [6]:
conn.autocommit=True

In [7]:
cur=conn.cursor()

In [17]:
def crt_credit_card():
    try:
        # CREDIT_CARD TABLE
        cur.execute("""
        CREATE TABLE IF NOT EXISTS healthcare_org.credit_card (
            patient_id VARCHAR(50) PRIMARY KEY,
            patient_firstname VARCHAR(50) NOT NULL,
            patient_lastname VARCHAR(50) NOT NULL,
            ssn VARCHAR(20) NOT NULL,
            contact VARCHAR(30) NULL,
            created TIMESTAMP DEFAULT CURRENT_TIMESTAMP
        );
        """)
        print("credit card tables created successfully...")
    except Exception as e:
        print(f"{e}")
crt_credit_card()

'psycopg2.extensions.cursor' object has no attribute 'excecute'


In [19]:
def crt_encounters():
    try:
        # ENCOUNTERS TABLE
        cur.execute("""
        CREATE TABLE IF NOT EXISTS healthcare_org.encounters (
            patient_id VARCHAR(50) PRIMARY KEY,
            date DATE,
            diagnosis TEXT,
            physician VARCHAR(150) NULL,
            cost DECIMAL(10,2) NULL,
            hospital VARCHAR(200) NULL,
            created TIMESTAMP DEFAULT CURRENT_TIMESTAMP
        );
        """)
        print("encounters tables created successfully...")
    except Exception as e:
        return f"{e}"
crt_encounters()

encounters tables created successfully...


In [21]:
def crt_insurance():
    try:
        # INSURANCE TABLE
        cur.execute("""
        CREATE TABLE IF NOT EXISTS healthcare_org.insurance (
            patient_id VARCHAR(50) PRIMARY KEY,
            insurance_id VARCHAR(100) NULL,
            Insurance_provider VARCHAR(50) NULL,
            insurance_coverage DECIMAL(10,2) NULL,
            created TIMESTAMP DEFAULT CURRENT_TIMESTAMP
        );
        """)
        print("insurance table created successfully...")
    except Exception as e:
        return f"{e}"
crt_insurance()

insurance table created successfully...


In [23]:
def crt_patients():
    try:
        # PATIENTS TABLE
        cur.execute("""
        CREATE TABLE IF NOT EXISTS healthcare_org.patients (
            patient_id VARCHAR(50) PRIMARY KEY,
            firstname VARCHAR(50) NOT NULL,
            lastname VARCHAR(50) NOT NULL,
            snn VARCHAR(20) NULL,
            contact VARCHAR(30) NULL,
            created TIMESTAMP DEFAULT CURRENT_TIMESTAMP
        );
        """)
        print("patients table created successfully...")
    except Exception as e:
        return f"{e}"
crt_patients()

patients table created successfully...


In [15]:
def load_to_credit_card():
    try:
        cur = conn.cursor()
        file_path = '/home/deathrok-xz70/Desktop/mariadb_test/healthcare_simulation/tasks/datasets/credit_card.csv'
        # Read CSV file
        print(f"Reading CSV file: {file_path}")
        df = pd.read_csv(file_path)
        print(f"Found {len(df)} records")
        # Get the exact column names from your CSV
        print(f"CSV columns: {list(df.columns)}")
        # Prepare data - convert NaN to None (NULL in SQL)
        df = df.where(pd.notnull(df), None)
        # For StringIO method - need to explicitly list columns in the same order as CSV
        # OPTION 1: Using StringIO (if columns match exactly)
        output = StringIO()
        df.to_csv(output, sep=',', header=False, index=False, quoting=csv.QUOTE_MINIMAL)
        output.seek(0)
        # Define columns explicitly in the order they appear in CSV
        csv_columns = list(df.columns)
        print(f"Loading {len(df.columns)} columns: {csv_columns}")
        # Use copy_from with StringIO
        cur.copy_from(
            output,
            'healthcare_org.credit_card',
            sep=',',
            null='',  # Empty strings will be treated as NULL
            columns=csv_columns
        )
        conn.commit()
        print(f"Successfully loaded {len(df)} records using COPY")
        
    except Exception as e:
        print(f"Error loading data: {e}")
        conn.rollback()
        import traceback
        traceback.print_exc()  # This will show the full error traceback
load_to_credit_card()

Reading CSV file: /home/deathrok-xz70/Desktop/mariadb_test/healthcare_simulation/tasks/datasets/credit_card.csv
Found 1000000 records
CSV columns: ['patient_id', 'patient_credit_card', 'patient_credit_card_expiry', 'patient_credit_card_cvv']
Loading 4 columns: ['patient_id', 'patient_credit_card', 'patient_credit_card_expiry', 'patient_credit_card_cvv']
Error loading data: relation "healthcare_org.credit_card" does not exist



Traceback (most recent call last):
  File "/tmp/ipykernel_9380/117816925.py", line 22, in load_to_credit_card
    cur.copy_from(
psycopg2.errors.UndefinedTable: relation "healthcare_org.credit_card" does not exist



In [24]:
def load_to_credit_card():
    try:
        cur = conn.cursor()
        file_path = '/home/deathrok-xz70/Desktop/mariadb_test/healthcare_simulation/tasks/datasets/credit_card.csv'
        
        # Use SQL COPY command
        copy_sql = """
            COPY healthcare_org.credit_card
            FROM STDIN
            WITH (FORMAT CSV, HEADER true, DELIMITER ',', NULL '');
        """
        
        with open(file_path, 'r') as f:
            cur.copy_expert(copy_sql, f)
        
        conn.commit()
        print("Successfully loaded data using COPY command")
        
    except Exception as e:
        print(f"Error: {e}")
        conn.rollback()
        import traceback
        traceback.print_exc()
load_to_credit_card()

Error: relation "healthcare_org.credit_card" does not exist



Traceback (most recent call last):
  File "/tmp/ipykernel_9380/4202900206.py", line 14, in load_to_credit_card
    cur.copy_expert(copy_sql, f)
psycopg2.errors.UndefinedTable: relation "healthcare_org.credit_card" does not exist



In [ ]:
def load_to_encounters():
    try:
        cur = conn.cursor()
        file_path = '/home/deathrok-xz70/Desktop/mariadb_test/healthcare_simulation/tasks/datasets/encounters.csv'
        # Read CSV file
        print(f"Reading CSV file: {file_path}")
        df = pd.read_csv(file_path)
        print(f"Found {len(df)} records")
        # Get the exact column names from your CSV
        print(f"CSV columns: {list(df.columns)}")
        # Prepare data - convert NaN to None (NULL in SQL)
        df = df.where(pd.notnull(df), None)
        # For StringIO method - need to explicitly list columns in the same order as CSV
        # OPTION 1: Using StringIO (if columns match exactly)
        output = StringIO()
        df.to_csv(output, sep=',', header=False, index=False, quoting=csv.QUOTE_MINIMAL)
        output.seek(0)
        # Define columns explicitly in the order they appear in CSV
        csv_columns = list(df.columns)
        print(f"Loading {len(df.columns)} columns: {csv_columns}")
        # Use copy_from with StringIO
        cur.copy_from(
            output,
            'healthcare-org-schema.encounters',
            sep=',',
            null='',  # Empty strings will be treated as NULL
            columns=csv_columns
        )
        conn.commit()
        print(f"Successfully loaded {len(df)} records using COPY")
        
    except Exception as e:
        print(f"Error loading data: {e}")
        conn.rollback()
        import traceback
        traceback.print_exc()  # This will show the full error traceback
load_encounters()

In [ ]:
def load_to_insurance():
    try:
        cur = conn.cursor()
        file_path = '/home/deathrok-xz70/Desktop/mariadb_test/healthcare_simulation/tasks/datasets/insurance.csv'
        # Read CSV file
        print(f"Reading CSV file: {file_path}")
        df = pd.read_csv(file_path)
        print(f"Found {len(df)} records")
        # Get the exact column names from your CSV
        print(f"CSV columns: {list(df.columns)}")
        # Prepare data - convert NaN to None (NULL in SQL)
        df = df.where(pd.notnull(df), None)
        # For StringIO method - need to explicitly list columns in the same order as CSV
        # OPTION 1: Using StringIO (if columns match exactly)
        output = StringIO()
        df.to_csv(output, sep=',', header=False, index=False, quoting=csv.QUOTE_MINIMAL)
        output.seek(0)
        # Define columns explicitly in the order they appear in CSV
        csv_columns = list(df.columns)
        print(f"Loading {len(df.columns)} columns: {csv_columns}")
        # Use copy_from with StringIO
        cur.copy_from(
            output,
            'healthcare-org-schema.insurance',
            sep=',',
            null='',  # Empty strings will be treated as NULL
            columns=csv_columns
        )
        conn.commit()
        print(f"Successfully loaded {len(df)} records using COPY")
        
    except Exception as e:
        print(f"Error loading data: {e}")
        conn.rollback()
        import traceback
        traceback.print_exc()  # This will show the full error traceback
load_to_insurance()

In [ ]:
def load_to_patients():
    try:
        cur = conn.cursor()
        file_path = '/home/deathrok-xz70/Desktop/mariadb_test/healthcare_simulation/tasks/datasets/patient.csv'
        # Read CSV file
        print(f"Reading CSV file: {file_path}")
        df = pd.read_csv(file_path)
        print(f"Found {len(df)} records")
        # Get the exact column names from your CSV
        print(f"CSV columns: {list(df.columns)}")
        # Prepare data - convert NaN to None (NULL in SQL)
        df = df.where(pd.notnull(df), None)
        # For StringIO method - need to explicitly list columns in the same order as CSV
        # OPTION 1: Using StringIO (if columns match exactly)
        output = StringIO()
        df.to_csv(output, sep=',', header=False, index=False, quoting=csv.QUOTE_MINIMAL)
        output.seek(0)
        # Define columns explicitly in the order they appear in CSV
        csv_columns = list(df.columns)
        print(f"Loading {len(df.columns)} columns: {csv_columns}")
        # Use copy_from with StringIO
        cur.copy_from(
            output,
            'healthcare-org-schema.patients',
            sep=',',
            null='',  # Empty strings will be treated as NULL
            columns=csv_columns
        )
        conn.commit()
        print(f"Successfully loaded {len(df)} records using COPY")
        
    except Exception as e:
        print(f"Error loading data: {e}")
        conn.rollback()
        import traceback
        traceback.print_exc()  # This will show the full error traceback
load_to_patients()